In [1]:
# mnist_cnn
# MNIST and Convolutional Neural Network
# L1,L2 : conv2d + relu + max_pool 
# L3 : FC(Fully Connected Layer)

import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
tf.random.set_seed(5)
tf.__version__

'2.17.0'

In [2]:
# MNSIT 데이터 가져오기
mnist = tf.keras.datasets.mnist 
(x_train,y_train),(x_test,y_test) = mnist.load_data()
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

(60000, 28, 28)
(60000,)
(10000, 28, 28)
(10000,)


In [3]:
# one-hot 인코딩
nb_classes = 10  # 분류 class의 갯수(0~9)

Y_one_hot = tf.one_hot(y_train,nb_classes)   
print(Y_one_hot.shape)                     # (60000, 10)

(60000, 10)


In [4]:
# X값의 타입을 float형으로 변환
x_train = tf.cast(x_train,dtype=tf.float32)
print(x_train.shape,x_train.dtype)

x_test = tf.cast(x_test,dtype=tf.float32)
print(x_test.shape,x_test.dtype)

(60000, 28, 28) <dtype: 'float32'>
(10000, 28, 28) <dtype: 'float32'>


In [5]:
# X값의 shape을 4차원으로 변환 : (N,H,W,C)
X_img = tf.reshape(x_train,[-1,28,28,1])
print(X_img.shape)  # (60000, 28, 28, 1)

(60000, 28, 28, 1)


In [6]:
# Layer 1 : conv2d - relu - max_pool
# (?, 28, 28, 1) --> (?, 14, 14, 32)

# <1> conv2d
# L1 input image shape : (?, 28, 28, 1)
# filter : (3,3,1,32), 필터 32개
# strides : (1,1,1,1), padding='SAME'
# 출력 이미지 : (28+2 - 3)/1 + 1 = 28
# (?, 28, 28, 1) --> (?, 28, 28, 32)
W1 = tf.Variable(tf.random.normal([3,3,1,32]), name='weight1')

def L1_conv2d(X):
    return tf.nn.conv2d(X,W1,strides=[1,1,1,1],padding='SAME')

# <2> relu
def L1_relu(X):
    return tf.nn.relu(L1_conv2d(X))     # shape 변화가 없다

# <3> max_pool
# input image : (?, 28, 28, 32)
# ksize : (1,2,2,1), strides : (1,2,2,1), padding='SAME'
# 출력 이미지 : (28+1 - 2)/2 + 1 = 14
# (?, 28, 28, 32) -->  (?, 14, 14, 32)
def L1_MaxPool(X):
    return tf.nn.max_pool(L1_relu(X),ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME')

In [7]:
# L1_MaxPool(X_img[:1])  # 출력 (?, 14, 14, 32)

In [8]:
# Layer 2 : conv2d - relu - max_pool
#  (?, 14, 14, 32) --> (?, 14, 14, 32)

# <1> conv2d
# L1 input image shape :  (?, 14, 14, 32)
# filter : (3,3,1,64), 필터 64개
# strides : (1,1,1,1), padding='SAME'
# 출력 이미지 : (14+2 - 3)/1 + 1 = 14
# (?, 14, 14, 32) --> (?, 14, 14, 64)
W2 = tf.Variable(tf.random.normal([3,3,1,64]), name='weight2')

def L2_conv2d(X):
    return tf.nn.conv2d(L1_MaxPool(X),W2,strides=[1,1,1,1],padding='SAME')

# <2> relu
def L2_relu(X):
    return tf.nn.relu(L2_conv2d(X))     # shape 변화가 없다

# <3> max_pool
# input image : (?, 14, 14, 64)
# ksize : (1,2,2,1), strides : (1,2,2,1), padding='SAME'
# 출력 이미지 : (14+1 - 2)/2 + 1 = 7
# (?, 14, 14, 64) -->  (?, 7, 7, 64)
def L2_MaxPool(X):
    return tf.nn.max_pool(L2_relu(X),ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME')

# <4> flatten layer : : 다차원 배열을 2차원으로 변환하여 FC layer에 전달한다
def L2_flat(X):
    return tf.reshape(L2_MaxPool(X),[-1,7*7*64])

In [9]:
# L2_MaxPool(X_img[:1])  # 출력 (?, 7, 7, 64)
# L2_flat(X_img[:1])  # 출력 (?, 7*7*64)  = (?, 3136)

In [10]:
# Layer 3 : FC (Fully Connected Layer)
# (?, 7*7*64) * (7*67*64,10) = (?,10)
W3 = tf.Variable(tf.random.normal([7*7*64,nb_classes]),name='weight3')
b = tf.Variable(tf.random.normal([nb_classes]),name='bias')    

In [11]:
# 예측 함수(hypothesis) : H(X) = softmax(X*W + b)
def logits(X):
    return tf.matmul(L2_flat(X),W3) + b
    
def hypothesis(X):
    return tf.nn.softmax(logits(X))  

In [12]:
# 비용 함수  : tf.nn.softmax_cross_entropy_with_logits() 함수 사용
# def cost_func():
#     cost_i = tf.nn.softmax_cross_entropy_with_logits(logits = logits(X_img),
#                                              labels = Y_one_hot)
#     cost =  tf.reduce_mean(cost_i)
#     return cost

In [13]:
# batch 사이즈로 나누어 학습, 효율적 이며 학습 시간 단축

training_epoch = 15  # 50
batch_size = 600     # 32

# 경사 하강법
# learning_rate(학습율)을 0.01 로 설정하여 optimizer객체를 생성
optimizer = tf.compat.v1.train.AdamOptimizer(learning_rate=0.01)

# 학습 시작  : mini batch 학습
print('****** Start Learning!!')

for epoch in range(training_epoch):  # 15회

    avg_cost = 0
    
    total_batch = int(x_train.shape[0]/batch_size)  # 60000/600=100
    for k in range(total_batch):
        batch_xs = x_train[0 + k*batch_size:batch_size  + k*batch_size]    # 600개의 X 데이터
        batch_ys = Y_one_hot[0 + k*batch_size:batch_size  + k*batch_size]  # 600개의 Y 데이터

        # X값의 shape을 4차원으로 변환
        X_img = tf.reshape(batch_xs,[-1,28,28,1])

        # 비용 함수  
        def cost_func_batch():
            cost_i = tf.nn.softmax_cross_entropy_with_logits(logits = logits(X_img),
                                                     labels = batch_ys)
            cost =  tf.reduce_mean(cost_i)
            return cost

        # cost를 minimize 한다
        optimizer.minimize(cost_func_batch,var_list=[W1,W2,W3,b])
        avg_cost += cost_func_batch().numpy()/total_batch
        
    print('Epoch:','%04d'%(epoch + 1),'cost:','{:.9f}'.format(avg_cost))
    
print('****** Learning Finished!!')        


****** Start Learning!!
Epoch: 0001 cost: 6351.685648193
Epoch: 0002 cost: 656.206498947
Epoch: 0003 cost: 368.835160303
Epoch: 0004 cost: 254.495237164
Epoch: 0005 cost: 188.864557824
Epoch: 0006 cost: 152.123358837
Epoch: 0007 cost: 124.895512800
Epoch: 0008 cost: 106.864025577
Epoch: 0009 cost: 92.738631496
Epoch: 0010 cost: 79.038738999
Epoch: 0011 cost: 69.791744175
Epoch: 0012 cost: 60.383924556
Epoch: 0013 cost: 53.556837530
Epoch: 0014 cost: 47.478008835
Epoch: 0015 cost: 43.876828144
****** Learning Finished!!


In [16]:
# 정확도 측정 : accuracy computation

# X값의 shape을 4차원으로 변환 : (N,H,W,C)
X_img = tf.reshape(x_test,[-1,28,28,1])
print(X_img.shape)  # (10000, 28, 28, 1)

# one-hot 인코딩
Y_one_hot = tf.one_hot(y_test,nb_classes)   
print(Y_one_hot.shape)                     # (10000, 10)

# tf.argmax() : 값이 가장 큰 요소의 인덱스 값을 반환
def predict(X):
    return tf.argmax(hypothesis(X),axis=1)
    
correct_predict = tf.equal(predict(X_img),tf.argmax(Y_one_hot,1))
accuracy = tf.reduce_mean(tf.cast(correct_predict, dtype = tf.float32))
print("Accuracy:",accuracy.numpy())

(10000, 28, 28, 1)
(10000, 10)
Accuracy: 0.9741


In [21]:
#예측 
print('***** Predict')
pred = predict(X_img[:10]).numpy()
print(pred,y_test)

***** Predict
[7 2 1 0 4 1 4 9 5 9] [7 2 1 ... 4 5 6]


In [ ]:
# 정확도 비교

# [1] softmax 사용
# 1 layers              -------> Accuracy  : 0.8871
# 4 layers  sigmoid     -------> Accuracy  : 0.9033
# 4 layers  relu        -------> Accuracy  : 0.9534  

# [2] CNN 사용
# 3 layers              -------> Accuracy  : 0.9764 (epoch=15) ,GPU
# 3 layers              -------> Accuracy  : 0.9804 (epoch=50) ,GPU